In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import math

df = pd.read_csv("/Users/swastikagarwal/Downloads/PERSONAL/sem 5 codes/NLP/Musical_instruments_reviews 4.csv")
X = df['summary']
# y = (df['overall'] >= 4).astype(int)
y=df['overall']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

stop_words = set(stopwords.words('english'))
preprocess = lambda text: ' '.join([word.lower() for word in word_tokenize(text) if word.isalpha() and word.lower() not in stop_words])
X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

# Manual TF-IDF calculation
def calculate_tf(word, sentence):
    words = sentence.split()
    return words.count(word) / (len(words)+1)

def calculate_idf(word, sentences):
        no = 0
        for sent in sentences:
            if word in sent:
                no += 1
        return math.log(len(sentences) / (no + 1))

# Calculate TF-IDF for training set
all_documents = X_train.tolist() + X_test.tolist()
idf_values = {}
unique_terms = set(' '.join(all_documents).split())

for term in unique_terms:
    idf_values[term] = calculate_idf(term, all_documents)


# Define the vocabulary based on the sorted terms
vocabulary = sorted(list(idf_values.keys()))

# Calculate TF-IDF for training set
X_train_tfidf_manual = []

for document in X_train:
    tfidf_vector = []
    for term in vocabulary:
        tfidf_value = calculate_tf(term, document) * idf_values[term]
        tfidf_vector.append(tfidf_value)
    X_train_tfidf_manual.append(tfidf_vector)

# Calculate TF-IDF for test set
X_test_tfidf_manual = []
for document in X_test:
    tfidf_vector = [calculate_tf(term, document) * idf_values[term] for term in vocabulary]
    X_test_tfidf_manual.append(tfidf_vector)

# Convert the lists to numpy arrays
X_train_tfidf_manual = np.array(X_train_tfidf_manual)
X_test_tfidf_manual = np.array(X_test_tfidf_manual)

# Train the model
model = LogisticRegression(max_iter=200).fit(X_train_tfidf_manual, y_train)


# Predict on new text
new_text = "It's bad"
new_text_processed = preprocess(new_text)
new_text_tfidf_manual = []
for term in vocabulary:
    tf = calculate_tf(term, new_text_processed)
    idf = idf_values[term]
    tf_idf = tf * idf
    new_text_tfidf_manual.append(tf_idf)
predicted_sentiment = model.predict([new_text_tfidf_manual])
# print("Predicted Sentiment:", "Positive" if predicted_sentiment[0] == 1 else "Negative")
print("Predicted Sentiment:",predicted_sentiment[0])

Accuracy: 0.6664501461513478
Predicted Sentiment: 3.0


In [2]:
print(X_test)


2507    great solution acoustic guitar
5159    crazy took long time arrive us
932                    would grab play
1190            works great easy carry
2619                       nice guitar
                     ...              
7307                    nice jackplate
5384                               get
7341                  good lead guitar
8921                   excellent tuner
3429                             works
Name: summary, Length: 3079, dtype: object


In [3]:
new_text = "crap."
new_text_tfidf_manual = [calculate_tf(term, preprocess_text(new_text)) * idf_values[term] for term in vocabulary]
predicted_sentiment = model.predict([new_text_tfidf_manual])
print("Predicted Sentiment:",predicted_sentiment[0])

NameError: name 'preprocess_text' is not defined